In [ ]:
import pandas as pd
import numpy as np

import torch
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

from datasets import load_dataset
from datasets import load_metric

from transformers import pipeline
from transformers import TrainingArguments, Trainer
from transformers import DeiTForImageClassification, DeiTImageProcessor, DeiTModel
from transformers import ViTForImageClassification 

from deit_blockprune_trainer import DeiTSparseTrainingArguments, DeiTModelPatchingCoordinator, DeiTSparseTrainer
from inference_patchers_blockprune import optimize_model

from PIL import Image

from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt

from time import perf_counter

from contextlib import redirect_stdout

import os

import yaml
import json
from pathlib import Path

In [ ]:
filepath = 'results_block_pruning_dense_remove/GPU_25_CPU_25.json'

with open(filepath) as json_file:
    result_dict = json.load(json_file)

print(result_dict) 

In [ ]:
plot_dict = {     8:   {'latency_GPU': [],
                        'latency_CPU': [],
                        'accuracy': [],
                        'threshold': [],
                        'density': []},

                 16:   {'latency_GPU': [],
                        'latency_CPU': [],
                        'accuracy': [],
                        'threshold': [],
                        'density': []},

                 32:   {'latency_GPU': [],
                        'latency_CPU': [],
                        'accuracy': [],
                        'threshold': [],
                        'density': []},

                 64:   {'latency_GPU': [],
                        'latency_CPU': [],
                        'accuracy': [],
                        'threshold': [],
                        'density': []},

             'base':   {'latency_GPU': [],
                        'latency_CPU': [],
                        'accuracy': [],
                        'threshold': [],
                        'density': []}}


for k, v in result_dict.items():
    if k != "baseline":
        plot_dict[v['block_size']]['threshold'].append(v['threshold'])
        plot_dict[v['block_size']]['latency_GPU'].append(v['GPU_latency_ms:'])
        plot_dict[v['block_size']]['latency_CPU'].append(v['CPU_latency_ms:'])
        plot_dict[v['block_size']]['accuracy'].append(v['model_accuracy'])
        plot_dict[v['block_size']]['density'].append(v['effective_density_BASELINE'])         # Using baseline NOT relative
    else:
        plot_dict['base']['threshold'].append(v['threshold'])
        plot_dict['base']['latency_GPU'].append(v['GPU_latency_ms:'])
        plot_dict['base']['latency_CPU'].append(v['CPU_latency_ms:'])
        plot_dict['base']['accuracy'].append(v['model_accuracy'])
        plot_dict['base']['density'].append(v['effective_density_BASELINE'])                 # Using baseline NOT relative

print(plot_dict)
         

In [ ]:
common_batch_size = result_dict['baseline']['batch_size_CPU'] if result_dict['baseline']['batch_size_CPU'] == result_dict['baseline']['batch_size_GPU'] else None

In [ ]:
batch_size_cpu = common_batch_size
batch_size_gpu = common_batch_size

In [ ]:
plt.plot(plot_dict[8]['accuracy'], plot_dict[8]['latency_CPU'], label='8 X 8')
plt.scatter(plot_dict[8]['accuracy'], plot_dict[8]['latency_CPU'], marker='o', s=[20, 40, 80, 140] if plot_dict[8]['threshold'] == [0.2, 0.3, 0.4, 0.5] else None)

plt.plot(plot_dict[16]['accuracy'], plot_dict[16]['latency_CPU'], label='16 X 16')
plt.scatter(plot_dict[16]['accuracy'], plot_dict[16]['latency_CPU'], marker='^', s=[20, 40, 80, 140] if plot_dict[16]['threshold'] == [0.2, 0.3, 0.4, 0.5] else None)

plt.plot(plot_dict[32]['accuracy'], plot_dict[32]['latency_CPU'], label='32 X 32')
plt.scatter(plot_dict[32]['accuracy'], plot_dict[32]['latency_CPU'], marker='*', s=[20, 40, 80, 140] if plot_dict[32]['threshold'] == [0.2, 0.3, 0.4, 0.5] else None)

plt.plot(plot_dict[64]['accuracy'], plot_dict[64]['latency_CPU'], label='64 X 64')
plt.scatter(plot_dict[64]['accuracy'], plot_dict[64]['latency_CPU'], marker='D', s=[20, 40, 80, 140] if plot_dict[64]['threshold'] == [0.2, 0.3, 0.4, 0.5] else None)

plt.plot(plot_dict['base']['accuracy'], plot_dict['base']['latency_CPU'], label='Baseline')
plt.scatter(plot_dict['base']['accuracy'], plot_dict['base']['latency_CPU'], marker='x', s=[300])

plt.legend()
plt.xlabel('Accuracy')
plt.ylabel('CPU Latency (ms)')
plt.title('CPU Latency VS Accuracy' + str(', Batch Size = ') + str(batch_size_cpu))
plt.savefig(os.path.join(*['plots', 'cpu_latency_batch_size_' + str(batch_size_cpu)]))
plt.show()

In [ ]:
plt.plot(plot_dict[8]['accuracy'], plot_dict[8]['latency_GPU'], label='8 X 8')
plt.scatter(plot_dict[8]['accuracy'], plot_dict[8]['latency_GPU'], marker='o', s=[20, 40, 80, 140] if plot_dict[8]['threshold'] == [0.2, 0.3, 0.4, 0.5] else None)

plt.plot(plot_dict[16]['accuracy'], plot_dict[16]['latency_GPU'], label='16 X 16')
plt.scatter(plot_dict[16]['accuracy'], plot_dict[16]['latency_GPU'], marker='^', s=[20, 40, 80, 140] if plot_dict[16]['threshold'] == [0.2, 0.3, 0.4, 0.5] else None)

plt.plot(plot_dict[32]['accuracy'], plot_dict[32]['latency_GPU'], label='32 X 32')
plt.scatter(plot_dict[32]['accuracy'], plot_dict[32]['latency_GPU'], marker='*', s=[20, 40, 80, 140] if plot_dict[32]['threshold'] == [0.2, 0.3, 0.4, 0.5] else None)

plt.plot(plot_dict[64]['accuracy'], plot_dict[64]['latency_GPU'], label='64 X 64')
plt.scatter(plot_dict[64]['accuracy'], plot_dict[64]['latency_GPU'], marker='D', s=[20, 40, 80, 140] if plot_dict[64]['threshold'] == [0.2, 0.3, 0.4, 0.5] else None)

plt.plot(plot_dict['base']['accuracy'], plot_dict['base']['latency_GPU'], label='Baseline')
plt.scatter(plot_dict['base']['accuracy'], plot_dict['base']['latency_GPU'], marker='x', s=[300])

plt.legend()
plt.xlabel('Accuracy')
plt.ylabel('GPU Latency (ms)')
plt.title('GPU Latency VS Accuracy' + str(', Batch Size = ') + str(batch_size_gpu))
plt.savefig(os.path.join(*['plots', 'GPU_latency_batch_size_' + str(batch_size_gpu)]))
plt.show()